<a href="https://colab.research.google.com/github/francoishcm/BackTesting/blob/master/hcmallza20240209_rev0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install packages

In [ ]:
!pip install yfinance

In [ ]:
!pip install yahooquery

In [ ]:
!pip install stockstats

In [ ]:
!pip install mpld3

###Create Folders in working directory

In [ ]:
import os

def create_folders(folders):
  cwd = os.getcwd()
  for folder in folders:
    path = os.path.join(cwd, folder)
    if not os.path.exists(path):
      os.mkdir(path)

folders =['Stocks','Stocks_final3','momentum_tickers','momentum_stocks','ranking_table','sma_data','buy_list']
create_folders(folders)

###Modules and settings

In [41]:
%matplotlib inline
import datetime
from datetime import date
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats
import mpld3
mpld3.enable_notebook()
"""
Settings
"""
portfolio_value = 155164.96
perf_last_month = 151728.55
start = "2020-01-01"
end = "2024-02-08"
ranking_day = "2024-02-07"
index_date = 'STXCAP20240208.csv'
ranking_table ="ranking_table20240208.csv"
portfolio = "portfolio20240208.csv"
current_portfolio = "portfolio20240208.csv"

# Storing today's date into a variable
today = date.today()
# Storing the specific date
trading_day = today
# Calculate percent difference since last month
perf_pct = (portfolio_value / perf_last_month) - 1
# Print performance, format as percent with two decimals.
print("{} - Last Month Result: {:.2%}".format(trading_day, perf_pct))
# Remember today's portfolio value for next month's calculation
#perf_last_month = portfolio_value

2024-02-09 - Last Month Result: 2.26%


###Import Market Data

In [32]:
#Import required libraries
import pandas as pd
import json
import requests
import glob
import os
from yahooquery import Ticker
from datetime import datetime

now = datetime.now().strftime("%Y-%m-%d")
data = yf.Ticker("STXCAP.JO")
index = data.history(start=start,  end=end)
resetindex = index.reset_index(inplace = True)

index['Date'] = pd.to_datetime(index['Date']).dt.strftime('%Y-%m-%d')

index = index.drop(columns=(['Open', 'High','Low','Volume','Dividends','Stock Splits']))

index.set_index("Date", inplace=True)

index.to_csv(index_date)
STX = pd.read_csv(index_date,index_col='Date', parse_dates=['Date'])
STX['SMA'] = STX['Close'].rolling(200).mean()
STX.plot()
STX['diff'] = STX['Close'] - STX['SMA']
print(STX.tail(1))

"""
params = {'access_key': 'e52cf3b93696352e880916f8c8adbf0c',
          'limit': 350}
api_result = requests.get('http://api.marketstack.com/v1/exchanges/XJSE/tickers', params)
api_response = api_result.json()
print(f"Exchange Name = {api_response['data']['name']}")
for ticker in api_response['data']['tickers']:
  print(f"{ticker['name']}: {ticker['symbol']}")


# Serializing json
json_object = json.dumps(api_response, indent=4)
# Writing to sample.json
with open("jse.json", "w") as outfile:
    outfile.write(json_object)
"""
# Opening JSON file
with open('jse.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)
df1 = pd.json_normalize(json_object["data"]["tickers"])

# dropping null value columns to avoid errors
# new data frame with split value columns
new = df1["symbol"].str.split(".", n = 1, expand = True)
# making separate first name column from new data frame
df1["ticker"]= new[0]
# Dropping old Name columns
df1.drop(columns =["has_intraday","has_eod"], inplace = True)
new.columns = new.columns.astype(str)
new.columns.values[1] = "tickers"
new['0'] + '.' + new['tickers']
new.replace("XJSE", "JO", inplace=True)

result = pd.concat([df1, new], axis=1, join='inner')
df2=result['0'] + '.' + result['tickers']
final = pd.concat([df1, df2], axis=1, join='inner')
final.columns.values[3] = "tickers"
final.columns = final.columns.astype(str)
#Dropping old Name columns
final.drop(columns =["name", "symbol","ticker"], inplace = True)
# Converting a specific Dataframe
# column to list using Series.tolist()
xjse = final["tickers"].tolist()
print("Converting tickers to list...")
print("Output dataframe..." )
print("Writing csv file..." )
print("Saving JSE data to disc...")
print("Saving csv files for each stock to disc...")
symbols = xjse
tickers = Ticker(symbols, asynchronous=True)
data = tickers.history(start= start, end=end, interval='1d')
data= data.drop(['adjclose','dividends','splits'],axis=1)


#Convert dataframe to csv and save to disc
final.to_csv("tickers.csv")
data.to_csv("jse_stocks.csv")

resetindex = data.reset_index(inplace = True)

for i, g in data.groupby('symbol'):
    g.to_csv('/content/Stocks/{}.csv'.format(i), header=True, index_label=False)

# merging the files
joined_files = os.path.join("/content/Stocks/", "*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# new dataframe with different column order
df_new = df.iloc[:, [1, 2, 3, 4, 5, 6, 0]]

Tickers = df_new["symbol"].tolist()
mylist = list(dict.fromkeys(Tickers))
df = pd.DataFrame (mylist, columns = ['tickers'])
df.to_csv("/content/tickers.csv")

#Change date format to Year-Month-Day
df_new['date'] =  pd.to_datetime(df_new['date']).dt.strftime('%Y-%m-%d')

df_new.set_index('date', inplace=True)

for i, g in df_new.groupby('symbol'):
    g.to_csv('/content/Stocks_final3/{}.csv'.format(i), header=True, index_label=False)

symbols = xjse
tickers = Ticker(symbols, asynchronous=True)
data = tickers.history(start=start, end=end, interval='1d')
data= data.drop(['adjclose','dividends','splits'],axis=1)

resetindex = data.reset_index(inplace = True)

# new dataframe with different column order
df_new = data.iloc[:, [1, 2, 3, 4, 5, 6, 0]]

Tickers = df_new["symbol"].tolist()
mylist = list(dict.fromkeys(Tickers))
df = pd.DataFrame (mylist, columns = ['tickers'])
df.columns = df.iloc[1]

df.to_csv("/content/Stocks_final3/tickers.csv")


csv_folder = Path('/content/Stocks_final3')

for file in csv_folder.glob('*.JO.csv'): #create iteratable & iterate on it.
    mydata = pd.read_csv(file)
    mydata.reset_index(inplace = True)
    mydata = mydata.rename(columns={'index': 'date'}, index=None)
    mydata['date'] =  pd.to_datetime(mydata['date']).dt.strftime('%Y-%m-%d')
    mydata.set_index('date', inplace=True)
    df =mydata[["open","high","low","close","volume"]]
    #no_outlier_prices = df[(np.abs(stats.zscore(df)) <2).all(axis=1)]
    new_file_name = file.parent.joinpath(f"{file.stem}.csv")
    df.to_csv(new_file_name) #, index=None, encoding='utf-8-sig', decimal=',')


/usr/local/lib/python3.10/dist-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


             Close          SMA       diff
Date                                      
2024-02-07  3716.0  3703.312454  12.687546
Converting tickers to list...
Output dataframe...
Writing csv file...
Saving JSE data to disc...
Saving csv files for each stock to disc...


/usr/local/lib/python3.10/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
/usr/local/lib/python3.10/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
/usr/local/lib/python3.10/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
/usr/local/lib/python3.10/dist-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "

###Indicators

In [38]:
%matplotlib inline
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
from pathlib import Path
import pandas as pd
import stockstats
from stockstats import StockDataFrame as sdf
import glob

plt.rcParams["figure.figsize"] = (12, 9) # (w, h)
plt.ioff()

tickers = pd.read_csv('/content/Stocks_final3/tickers.csv', header=None)[1].tolist()
stocks = (
    (pd.concat(
        [pd.read_csv(f"/content/Stocks_final3/{ticker}.csv",parse_dates=None, index_col=0)[
        'close'].rename(ticker)
        for ticker in tickers],
        axis=1,
        sort=True)
    )
)
stocks = stocks.loc[:,~stocks.columns.duplicated()]

from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2

momentums = stocks.copy(deep=True)
for ticker in tickers:
    momentums[ticker] = stocks[ticker].rolling(90).apply(momentum, raw=False)

result = momentums.sort_values(by =ranking_day, axis = 1, ascending =False)
result.reset_index(inplace=True)
res=result[~(result['date'] < ranking_day)].dropna(axis=1)

mes = res.reindex(columns = res.columns.tolist()
                                  + ['empty'])

df = mes.set_index(['empty']).stack().reset_index(name='index').rename(columns={'level_2':'date'})

df.columns = df.iloc[0]

df.index = list(df.index)

df.columns.values[0] = "empty"
df.columns.values[1] = "symbol"
df.columns.values[2] = "momentum_score"

df=df.drop(['empty'], axis=1)

finalframe=df.drop(index=0)

finalframe['momentum_score'] =finalframe['momentum_score'].astype(float)

finalframe['rank']=finalframe['momentum_score'].rank(ascending=False)

finalframe['momentum_score'] =finalframe['momentum_score'].astype(float)

#finalframe.style.format({'momentum_score':"{:.2f}"})

Mylist = finalframe["symbol"].tolist()

finalframe.set_index('symbol',inplace=True)

final = pd.concat([df1, df2], axis=1, join='inner')
final.columns.values[3] = "tickers"
final.columns = final.columns.astype(str)

final.set_index('tickers', inplace=True)
Yes = final.reindex(Mylist)

rt = pd.concat([finalframe, Yes], axis=1, join='inner')

rt = rt.iloc[:, [2, 3, 4, 0, 1]]

#rt.style.format({'momentum_score':"{:.2f}"})

# get names of indexes for which column "momentum_score" has value > 40%
index_names = rt[(rt['momentum_score'] < 0.001)].index

# drop these given row indexes from dataFrame
rt.drop(index_names, inplace = True)

rt.reset_index(inplace=True)

lowranked = rt [( rt ['rank'] >= 71)].index

# drop these given row indexes from dataFrame
rt.drop( lowranked, inplace = True)

My_final_list = rt["index"].tolist()

rt.set_index('index',inplace=True)

rt.to_csv(ranking_table)
final_table_columns = My_final_list
final_table =stocks.drop(columns=[col for col in stocks if col not in final_table_columns])

for column in final_table.columns:
    final_table[column].to_csv('/content/momentum_tickers/' + column + '.csv')

vola_table = Path('/content/momentum_tickers')

for file in vola_table.glob('*.JO.csv'): #create iteratable & iterate on it.
    vola_df = pd.read_csv(file)
    vola_df.columns.values[1] = "close"
    vola_df["20d_vol"] = vola_df["close"].pct_change().rolling(20).std()
    #vola_df = sdf.retype(vola_df)['close_90_roc']
    vola_df = pd.DataFrame(vola_df)
    vola_df['date'] =  pd.to_datetime(vola_df['date']).dt.strftime('%Y-%m-%d')
    vola_df['close_90_roc'] = (vola_df['close'] / vola_df['close'].shift(1) - 1).fillna(0)
    vola_df.set_index('date', inplace=True)
    #stockstats_df = sdf.retype(vola_df)['close_90_roc']
    #df = pd.DataFrame(stockstats_df)
    vola_file_name= file.parent.joinpath(f"/content/ranking_table/{file.stem}.csv")
    vola_df.to_csv(vola_file_name) #, index=None, encoding='utf-8-sig', decimal=',')

vola_table1 = Path('/content/ranking_table')

for file in vola_table1.glob('*.JO.csv'): #create iteratable & iterate on it.
    vola_df1 = pd.read_csv(file).tail(1)

    vola_df1['date'] =  pd.to_datetime(vola_df1['date']).dt.strftime('%Y-%m-%d')

    vola_df1.set_index('date', inplace=True)

    vola_file_name1= file.parent.joinpath(f"/content/momentum_stocks/{file.stem}.csv")

    vola_df1.to_csv(vola_file_name1)


# set search path and glob for files
# here we want to look for csv files in the input directory
path = '/content/'
files = glob.glob(path + '/momentum_stocks/*.csv')

# create empty list to store dataframes
li = []

# loop through list of files and read each one into a dataframe and append to list
for f in files:
    # get filename
    stock = os.path.basename(f)
    # read in csv
    temp_df = pd.read_csv(f)
    # create new column with filename
    temp_df['ticker'] = stock
    # data cleaning to remove the .csv
    temp_df['ticker'] = temp_df['ticker'].replace('.csv', '', regex=True)
    # append df to list
    li.append(temp_df)
    #print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

# concatenate our list of dataframes into one!
df = pd.concat(li)

ranking_list =df.set_index('ticker').reindex(My_final_list).reset_index()

inv_vol = 1 / ranking_list["20d_vol"]
sum_inv_vol=inv_vol.sum()
vol_targets = inv_vol / sum_inv_vol
ranking_list["weight"] = vol_targets

ranking_list.set_index('ticker', inplace=True)

final_list = pd.concat([rt,ranking_list], axis=1, join='inner')

stockprice = final_list["close"] / 100
pos_weight = portfolio_value * final_list["weight"]
final_list["units"] = pos_weight / stockprice
final_list["ZAR"] = stockprice * final_list["units"]


sma = Path('/content/ranking_table')

for file in sma.glob('*.JO.csv'): #create iteratable & iterate on it.
    sma = pd.read_csv(file)
    sma['date'] =  pd.to_datetime(sma['date']).dt.strftime('%Y-%m-%d')
    sma.set_index('date', inplace=True)

    '''Function'''

    sma['100d_sma'] = sma['close'].rolling(100).mean()

    #sma['index_sma']=index['Close'].rolling(200).mean()

    #sma['trend'] = np.where(sma['close'] > sma['100d_sma'], True, False)

    sma['index'] = index['Close']

    sma['index_sma']=index['Close'].rolling(200).mean()

    sma['trend'] = np.where(sma['close'] > sma['100d_sma'], True, False)

    sma['index_trend']= np.where(sma['index'] > sma['index_sma'], True, False)

    sma_file_name= file.parent.joinpath(f"/content/sma_data/{file.stem}.csv")

    sma.to_csv(sma_file_name)

buy_table = Path('/content/sma_data')

for file in buy_table.glob('*.JO.csv'): #create iteratable & iterate on it.
    buy_list_df = pd.read_csv(file).tail(1)

    buytable = file.parent.joinpath(f"/content/buy_list/{file.stem}.csv")

    buy_list_df.to_csv(buytable)

<ipython-input-38-734c7c1b6842>:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result.reset_index(inplace=True)


###Initial Portfolio and Ranking Table

In [39]:
import pandas as pd
import glob

# set search path and glob for files
# here we want to look for csv files in the input directory
path1 = '/content/'
files1 = glob.glob(path1 + '/buy_list/*.csv')

# create empty list to store dataframes
li1 = []

# loop through list of files and read each one into a dataframe and append to list
for f1 in files1:
    # get filename
    stock1 = os.path.basename(f1)
    # read in csv
    temp_df1 = pd.read_csv(f1)
    # create new column with filename
    temp_df1['ticker'] = stock1
    # data cleaning to remove the .csv
    temp_df1['ticker'] = temp_df1['ticker'].replace('.csv', '', regex=True)
    # append df to list
    li1.append(temp_df1)
    #print(f'Successfully created dataframe for {stock1} with shape {temp_df1.shape}')

# concatenate our list of dataframes into one!
Buy = pd.concat(li1)

buying_list = Buy.set_index('ticker').reindex(My_final_list)#.reset_index()

final_buy_list = pd.concat([final_list, buying_list], axis=1, join='inner')

#define list of columns range
cols = [x for x in range(final_buy_list.shape[1])]

#remove second column in DataFrame
cols.remove(12)
cols.remove(13)
cols.remove(14)
cols.remove(15)
cols.remove(16)#view resulting DataFrame
final_buy_list2 =final_buy_list.iloc[:, cols]
final_buy_list2.info()

final_buy_list2 = final_buy_list2.loc[final_buy_list2.trend, :]

index_names1 = final_buy_list2[(final_buy_list2['close_90_roc'] >= 15)].index

# drop these given row indexes from dataFrame
final_buy_list2.drop(index_names1, inplace = True)
final_buy_list2.to_csv(portfolio)
#final_buy_list2
'''
final_buy_list2.ticker = final_buy_list2.ticker+ '.JO'
Tickers = final_buy_list2["ticker"].tolist()
sector_list=[]

for tik in Tickers:
    ticker=yf.Ticker(tik)
    sector=ticker.info['sector']
    sector_list.append(sector)

final_buy_list2['sector'] = sector_list
'''
rp_units= final_buy_list2["units"].sum()
rp_weights = final_buy_list2["weight"].sum()
rp_holdings_value= final_buy_list2["ZAR"].sum()
cash_holdings_rp = portfolio_value - rp_holdings_value

print(f"FRP total units = {rp_units}")
print(f"FRP total units = {rp_weights}")
print(f"FRP holdings value = {rp_holdings_value}")
print(f"RP cash value = {cash_holdings_rp}")
pct_cash_rp = cash_holdings_rp / portfolio_value

final_buy_list2

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, LNF.JO to FFB.JO
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            70 non-null     object 
 1   symbol          70 non-null     object 
 2   ticker          70 non-null     object 
 3   momentum_score  70 non-null     float64
 4   rank            70 non-null     float64
 5   date            70 non-null     object 
 6   close           70 non-null     float64
 7   20d_vol         70 non-null     float64
 8   close_90_roc    70 non-null     float64
 9   weight          70 non-null     float64
 10  units           70 non-null     float64
 11  ZAR             70 non-null     float64
 12  100d_sma        70 non-null     float64
 13  index           70 non-null     float64
 14  index_sma       70 non-null     float64
 15  trend           70 non-null     bool   
 16  index_trend     70 non-null     bool   
dtypes: bool(2), float64(11), object(4

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
LNF.JO,LONDON FIN INV GROUP PLC,LNF.XJSE,LNF,3617.264451,1.0,2024-02-07,800.0,22.149218,0.000000,0.014285,277.072070,2216.576559,131.6240,3716.0,3703.312454,True,True
MUR.JO,MURRAY & ROBERTS HLDGS,MUR.XJSE,MUR,809.520314,2.0,2024-02-07,119.0,22.697433,-0.008333,0.013940,1817.679949,2163.039139,9.2479,3716.0,3703.312454,True,True
MIX.JO,MIX TELEMATICS LTD,MIX.XJSE,MIX,337.139595,3.0,2024-02-07,716.0,22.007275,-0.032432,0.014377,311.574451,2230.873069,54.7475,3716.0,3703.312454,True,True
EPS.JO,EASTERN PLATINUM LTD,EPS.XJSE,EPS,317.923649,4.0,2024-02-07,155.0,22.142874,0.000000,0.014289,1430.459079,2217.211572,12.0803,3716.0,3703.312454,True,True
RHB.JO,RH BOPHELO LIMITED,RHB.XJSE,RHB,264.757155,5.0,2024-02-07,400.0,23.625072,0.000000,0.013393,519.526844,2078.107378,26.4405,3716.0,3703.312454,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HDC.JO,HUDACO INDUSTRIES LTD,HDC.XJSE,HDC,65.319021,66.0,2024-02-07,16500.0,22.965859,0.024845,0.013777,12.956106,2137.757485,1282.6911,3716.0,3703.312454,True,True
EXX.JO,EXXARO RESOURCES LTD,EXX.XJSE,EXX,65.096618,67.0,2024-02-07,18088.0,21.791702,-0.007354,0.014520,12.455450,2252.941808,1462.9457,3716.0,3703.312454,True,True
DCP.JO,DIS-CHEM PHARMACIES LTD,DCP.XJSE,DCP,64.910643,68.0,2024-02-07,2995.0,22.159323,-0.010898,0.014279,73.975482,2215.565694,207.1367,3716.0,3703.312454,True,True
ARL.JO,ASTRAL FOODS LTD,ARL.XJSE,ARL,64.737322,69.0,2024-02-07,15575.0,22.209130,-0.001603,0.014247,14.193239,2210.596967,1226.5647,3716.0,3703.312454,True,True


###List to Liquidate current positions

In [42]:

sell_list = final_buy_list [~final_buy_list
 ["trend"]]
#sell_list.info()
dqs = sell_list ["ticker"].tolist()
cpf=pd.read_csv(current_portfolio)
#sell_list
liquidate = cpf.loc[cpf['ticker'].isin (dqs)].reset_index (drop=True)

newpf = cpf[~cpf['ticker'].isin(liquidate['ticker'])]
dfnew = newpf[~newpf['ticker'].isin(final_buy_list2['ticker'])]

remove = dfnew ["ticker"].tolist()
now = newpf[~newpf['ticker'].isin(dfnew['ticker'])]
now.set_index("ticker", inplace=True)
rebalance = now ["symbol"].tolist()

print(f"Liquidate disqualified stocks = {liquidate['name']}")
print(f"Liquidate stocks not in ranking table= {dfnew['name']}")
print(f"Portfolio after liquidation= {now['name']}")

Liquidate disqualified stocks = Series([], Name: name, dtype: object)
Liquidate stocks not in ranking table= Series([], Name: name, dtype: object)
Portfolio after liquidation= ticker
LNF    LONDON FIN INV GROUP PLC
MUR      MURRAY & ROBERTS HLDGS
MIX          MIX TELEMATICS LTD
EPS        EASTERN PLATINUM LTD
RHB          RH BOPHELO LIMITED
                 ...           
HDC       HUDACO INDUSTRIES LTD
EXX        EXXARO RESOURCES LTD
DCP     DIS-CHEM PHARMACIES LTD
ARL            ASTRAL FOODS LTD
FFB         FORTRESS REIT LTD B
Name: name, Length: 70, dtype: object


In [43]:
cpf

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,LNF.JO,LONDON FIN INV GROUP PLC,LNF.XJSE,LNF,3617.264451,1.0,2024-02-07,800.0,22.149218,0.000000,0.014285,277.072070,2216.576559,131.6240,3716.0,3703.312454,True,True
1,MUR.JO,MURRAY & ROBERTS HLDGS,MUR.XJSE,MUR,809.520314,2.0,2024-02-07,119.0,22.697433,-0.008333,0.013940,1817.679949,2163.039139,9.2479,3716.0,3703.312454,True,True
2,MIX.JO,MIX TELEMATICS LTD,MIX.XJSE,MIX,337.139595,3.0,2024-02-07,716.0,22.007275,-0.032432,0.014377,311.574451,2230.873069,54.7475,3716.0,3703.312454,True,True
3,EPS.JO,EASTERN PLATINUM LTD,EPS.XJSE,EPS,317.923649,4.0,2024-02-07,155.0,22.142874,0.000000,0.014289,1430.459079,2217.211572,12.0803,3716.0,3703.312454,True,True
4,RHB.JO,RH BOPHELO LIMITED,RHB.XJSE,RHB,264.757155,5.0,2024-02-07,400.0,23.625072,0.000000,0.013393,519.526844,2078.107378,26.4405,3716.0,3703.312454,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HDC.JO,HUDACO INDUSTRIES LTD,HDC.XJSE,HDC,65.319021,66.0,2024-02-07,16500.0,22.965859,0.024845,0.013777,12.956106,2137.757485,1282.6911,3716.0,3703.312454,True,True
66,EXX.JO,EXXARO RESOURCES LTD,EXX.XJSE,EXX,65.096618,67.0,2024-02-07,18088.0,21.791702,-0.007354,0.014520,12.455450,2252.941808,1462.9457,3716.0,3703.312454,True,True
67,DCP.JO,DIS-CHEM PHARMACIES LTD,DCP.XJSE,DCP,64.910643,68.0,2024-02-07,2995.0,22.159323,-0.010898,0.014279,73.975482,2215.565694,207.1367,3716.0,3703.312454,True,True
68,ARL.JO,ASTRAL FOODS LTD,ARL.XJSE,ARL,64.737322,69.0,2024-02-07,15575.0,22.209130,-0.001603,0.014247,14.193239,2210.596967,1226.5647,3716.0,3703.312454,True,True


###Update Ranking Table

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
portfolio_value2 = 156560.00

final_weights = final_buy_list
final_weights.set_index('rank').reset_index()
final_w = [x for x in range(final_weights.shape[1])]

final_w.remove(12)
final_w.remove(13)
final_w.remove(14)
final_w.remove(15)
final_w.remove(16)
final_weights2=final_weights.iloc[:, cols]
inv_vol2 = 1 / final_weights2["20d_vol"]
sum_inv_vol2 =inv_vol2.sum()
vol_targets2 = inv_vol2 / sum_inv_vol2
final_weights2["weight"] = vol_targets2

final_weights2.set_index('symbol', inplace=True)

stockprice2 = final_weights2["close"] / 100
pos_weight2 = portfolio_value2 * final_weights2["weight"]
final_weights2["units"] = pos_weight2 / stockprice2
final_weights2["ZAR"] = stockprice2 * final_weights2["units"]
final_weights2.to_csv(ranking_table)
final_weights2

###Rebalance Positions

In [ ]:
rebalance1 =final_weights2[final_weights2['name'].isin(now['name'])]

index_names4 = rebalance1 [( rebalance1 ['rank'] >= 71)].index

# drop these given row indexes from dataFrame
Allsale =rebalance1.drop(index_names4)

rebalance3 = rebalance1.reindex(rebalance).set_index('ticker')


now.reset_index(inplace=True)
rebalance3.reset_index(inplace=True)

rebalance3 ["bs"] = rebalance3["units"]-now["units"]
index_names5 = rebalance3 [( rebalance3 ['rank'] >= 71)].index

# drop these given row indexes from dataFrame
rebalance3.drop(index_names5, inplace = True)
rebalance3

###Enter New Positions

In [ ]:
ranking_table1 = final_weights2.reindex().set_index('ticker')
#ranking_table1["bs"] = ranking_table1["units"] * 1
ranking_table1.reset_index(inplace=True)

ranking_table1 = ranking_table1.loc[ranking_table1.trend, :]
index_names2 = ranking_table1 [( ranking_table1 ['close_90_roc'] >= 15)].index

# drop these given row indexes from dataFrame
ranking_table1.drop(index_names2, inplace = True)

buynew = ranking_table1 [~ ranking_table1 ['ticker'].isin( rebalance3 ['ticker'])]
index_names2 = ranking_table1 [( ranking_table1 ['close_90_roc'] >= 15)].index

# drop these given row indexes from dataFrame
ranking_table1.drop(index_names2, inplace = True)

buynamesnew2 = buynew [( buynew ['rank'] >= 71)].index

# drop these given row indexes from dataFrame
buynew.drop( buynamesnew2, inplace = True)

buynew

###Portfolio after position rebalance

In [ ]:
rebalancex = rebalance3.reset_index().set_index("level_0")
buynewx = buynew.reset_index().set_index("level_0")
new_portfolio2 = rebalancex.append(buynewx, ignore_index=True)
new_portfolio2.drop(columns =["bs"], inplace = True)

rslt_df = new_portfolio2.sort_values(by = 'rank', ascending = True)
'''
rslt_df .ticker = rslt_df.ticker+ '.JO'
Tickers = rslt_df["ticker"].tolist()
sector_list=[]

for tik in Tickers:
    ticker=yf.Ticker(tik)
    sector=ticker.info['sector']
    sector_list.append(sector)

rslt_df ['sector'] = sector_list
'''
frp_units=rslt_df["units"].sum()
frp_weights = rslt_df["weight"].sum()
frp_holdings_value=rslt_df["ZAR"].sum()
cash_holdings_frp = portfolio_value2 - frp_holdings_value
pct_cash_frp = cash_holdings_frp / portfolio_value2

print(f"FRP total units = {frp_units}")
print(f"FRP total units = {frp_weights}")
print(f"FRP holdings value = {frp_holdings_value}")
print(f"FRP cash value = {cash_holdings_frp}")

rslt_df

###Portfolio after rebalance

In [ ]:

Xxx = now[now['name'].isin(final_buy_list2['name'])]
Xxx2 = Xxx.reset_index().set_index("level_0")
buynewx = buynew.reset_index().set_index("level_0")
new_portfolio3 = Xxx2.append(buynewx, ignore_index=True)
rslt_df2 = new_portfolio3.sort_values(by = 'rank', ascending = True)
inv_vol = 1 / rslt_df2["20d_vol"]
rslt_df2["weight"] = rslt_df2 ["ZAR"] / portfolio_value2
rslt_df2.set_index('rank', inplace=True)
rslt_df2.drop(columns =["symbol"], inplace = True)
'''

rslt_df2 .ticker = rslt_df2.ticker+ '.JO'
Tickers = rslt_df2["ticker"].tolist()
sector_list=[]

for tik in Tickers:
    ticker=yf.Ticker(tik)
    sector=ticker.info['sector']
    sector_list.append(sector)

rslt_df2 ['sector'] = sector_list
'''
rp_units=rslt_df2["units"].sum()
rp_weights = rslt_df2["weight"].sum()
rp_holdings_value=rslt_df2["ZAR"].sum()
cash_holdings_rp = portfolio_value2 - rp_holdings_value

print(f"FRP total units = {rp_units}")
print(f"FRP total units = {rp_weights}")
print(f"FRP holdings value = {rp_holdings_value}")
print(f"RP cash value = {cash_holdings_rp}")
pct_cash_rp = cash_holdings_rp / portfolio_value2

rslt_df2

#Test

In [ ]:
final_buy_list2.ticker = final_buy_list2.ticker+ '.JO'
Tickers = final_buy_list2["ticker"].tolist()

In [ ]:
import yfinance as yf
import pandas as pd

df = pd.DataFrame()
for stock in Tickers :
  info = yf.Ticker(stock).info
  #industry = info.get('industry')
  #beta = info.get('beta')
  sector = info.get('sector')
  df = df.append({'Stock':stock, 'Sector':sector}, ignore_index=True)

df

In [ ]:
final_buy_list2.set_index('ticker', inplace=True)
df.set_index('Stock', inplace=True)
f_column = df["Sector"]
data1 = pd.concat([final_buy_list2 ,df], axis = 1)
data1

In [ ]:
data1['weight'].sum()

In [ ]:
new_dataset = data1.loc[:, ['Sector','weight']]
#new_dataset.set_index("Sector",inplace=True)
new_dataset.loc[len(new_dataset.index)] = ['Cash', pct_cash_frp]
new_dataset

In [ ]:
new_dataset["weight"].sum()

In [ ]:
new_dataset.groupby(['Sector']).sum().plot(kind='pie', y='weight', autopct='%1.0f%%')

plt.tight_layout()

# We save the graph in jpg format
plt.savefig("my_graph.jpg")
plt.show()

In [ ]:

dataset = new_dataset
# plot the dataset, referencing dataframe column names
import altair as alt
alt.Chart(dataset).mark_bar().encode(
  x='sum(weight)',
  y='Sector',
  color='Sector'
)

In [ ]:
RankingTable08092023 = pd.read_csv("/content/ranking_table130923.csv")
RankingTable08092023

In [ ]:
Portfolio = pd.read_csv("/content/portfolio130923.csv")
Portfolio

In [34]:
ABG_2 = pd.read_csv("/content/Stocks/ABG.JO.csv")
ABG_2

,symbol,date,open,high,low,close,volume
17657,ABG.JO,2020-01-02,150.850006,153.179993,149.889999,150.690002,942921.0
17658,ABG.JO,2020-01-03,148.500000,150.270004,146.639999,149.610001,1394666.0
17659,ABG.JO,2020-01-06,150.000000,150.000000,144.660004,145.360001,1185720.0
17660,ABG.JO,2020-01-07,147.000000,148.309998,141.580002,141.580002,5625419.0
17661,ABG.JO,2020-01-08,141.490005,143.399994,139.589996,142.949997,3584756.0
...,...,...,...,...,...,...,...
18679,ABG.JO,2024-02-01,16100.000000,16315.000000,15990.000000,16243.000000,3121103.0
18680,ABG.JO,2024-02-02,16285.000000,16575.000000,16203.000000,16284.000000,2455250.0
18681,ABG.JO,2024-02-05,16399.000000,16581.000000,16181.000000,16284.000000,1674321.0
18682,ABG.JO,2024-02-06,16500.000000,16537.000000,16300.000000,16421.000000,1358386.0


In [35]:
ABG = pd.read_csv("/content/Stocks_final3/ABG.JO.csv")
ABG

,date,open,high,low,close,volume
0,2020-01-02,150.850006,153.179993,149.889999,150.690002,942921.0
1,2020-01-03,148.500000,150.270004,146.639999,149.610001,1394666.0
2,2020-01-06,150.000000,150.000000,144.660004,145.360001,1185720.0
3,2020-01-07,147.000000,148.309998,141.580002,141.580002,5625419.0
4,2020-01-08,141.490005,143.399994,139.589996,142.949997,3584756.0
...,...,...,...,...,...,...
1022,2024-02-01,16100.000000,16315.000000,15990.000000,16243.000000,3121103.0
1023,2024-02-02,16285.000000,16575.000000,16203.000000,16284.000000,2455250.0
1024,2024-02-05,16399.000000,16581.000000,16181.000000,16284.000000,1674321.0
1025,2024-02-06,16500.000000,16537.000000,16300.000000,16421.000000,1358386.0
